In [21]:
import numpy as np
from numpy import linalg as LA
from numpy import matlib
from numpy.linalg import matrix_power
from timeit import default_timer as timer

In [22]:
D = 1000
U = np.zeros((D, 9))
V = np.zeros((D, 9))

In [23]:
for i in range(0, 9):
    U[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);
    V[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);

In [24]:
A = np.matmul(U,V.transpose())
eigenvalues, eigenvectors = LA.eig(A)
max_eigenvalue = max(np.absolute(eigenvalues))
A = np.dot(0.8, np.divide(A, max_eigenvalue))

In [25]:
mu = np.random.rand(D,1)/D
w = 1

In [26]:
import math
def comp_lambda(cur_t, cur_event, last_t, lambdat,w,mu,A):
    lambda_comp = mu + (lambdat - mu)*(math.exp(-w * (cur_t - last_t)))
    if (cur_event):
        lambda_comp = lambda_comp + np.expand_dims(A[cur_event, :].T,axis=1)
    return lambda_comp

In [27]:
import random 

def generate_samples(w, mu, A, num_sequences, max_events_per_sequence):
    start_time = timer()
    hp_samples = []

    for i in range(1, num_sequences+1):
        t = 0
        timestamp_and_event = []
        lambdat = mu
        lambdat_sum = np.sum(lambdat)
        
        while len(timestamp_and_event) < max_events_per_sequence:
            rand_u = random.uniform(0, 1)
            dt = np.random.exponential(1/lambdat_sum)            
            lambda_ts = comp_lambda(t+dt, [], t, lambdat,w,mu,A);
            lambdats_sum = np.sum(lambda_ts);
                        
            if (rand_u > (lambdats_sum/lambdat_sum)):
                t = t+dt
                lambdat = lambda_ts

            else:
                u = random.uniform(0, 1) * lambdats_sum
                lambda_sum = 0
                
                d = 0
                for d in range(1, len(mu)):
                    lambda_sum = lambda_sum + lambda_ts[d]
                    if(lambda_sum >= u):
                        break
            
                lambdat = comp_lambda(t+dt, d, t, lambdat, w, mu, A)
                t = t+dt
                timestamp_and_event.append([t,d])

            lambdat_sum = np.sum(lambdat)
        
        hp_samples.append(timestamp_and_event[0:])
        
        if (i%10 == 0):
            print("samples = " + str(i)+ "/" + str(num_sequences) + ", time = " 
                  + "{:.2f}".format(timer() - start_time) + " sec.")
    
    return hp_samples

In [28]:
num_sequences = 250
max_events_per_sequence = 100
hawkes_process_samples = generate_samples(w, mu, A, num_sequences, max_events_per_sequence)

samples = 10/250, time = 0.90 sec.
samples = 20/250, time = 1.78 sec.
samples = 30/250, time = 2.65 sec.
samples = 40/250, time = 4.11 sec.
samples = 50/250, time = 5.06 sec.
samples = 60/250, time = 6.01 sec.
samples = 70/250, time = 6.99 sec.
samples = 80/250, time = 7.92 sec.
samples = 90/250, time = 9.06 sec.
samples = 100/250, time = 9.96 sec.
samples = 110/250, time = 10.87 sec.
samples = 120/250, time = 11.77 sec.
samples = 130/250, time = 12.69 sec.
samples = 140/250, time = 13.50 sec.
samples = 150/250, time = 14.43 sec.
samples = 160/250, time = 15.21 sec.
samples = 170/250, time = 16.06 sec.
samples = 180/250, time = 16.94 sec.
samples = 190/250, time = 17.89 sec.
samples = 200/250, time = 18.77 sec.
samples = 210/250, time = 19.64 sec.
samples = 220/250, time = 20.49 sec.
samples = 230/250, time = 21.35 sec.
samples = 240/250, time = 22.35 sec.
samples = 250/250, time = 23.20 sec.


In [29]:
'''
timestamp = 1x100 double
event = 1x100 event            
'''
print (len(hawkes_process_samples))
print (len(hawkes_process_samples[0]))
print (hawkes_process_samples[0])

250
100
[[1.2917243665561897, 386], [3.55047049165047, 892], [3.6246631778461387, 729], [3.9332620237293368, 717], [3.9342697727426645, 869], [4.015494024934629, 699], [4.163497420014593, 754], [4.666554063512556, 866], [4.707987877460693, 731], [4.874313617908396, 806], [6.122493366267486, 839], [6.455257903224882, 871], [7.350884155009851, 77], [8.510632677229308, 121], [9.011560478442544, 819], [9.27480451265427, 217], [9.375526413119767, 235], [9.439732556262657, 938], [9.656890349908377, 923], [10.928031740070274, 790], [11.904350777157266, 98], [12.49808834042902, 435], [13.081871703116782, 416], [13.460101676054668, 454], [13.682775263001673, 378], [14.830971600342224, 371], [14.8502235953997, 489], [15.122438241901673, 453], [15.254721705701336, 431], [16.567363353563614, 901], [17.224146905524883, 897], [17.252415960312735, 261], [17.257835911849863, 829], [17.466266147100004, 802], [18.001221701507266, 805], [18.410571052457488, 820], [19.612926651643786, 778], [19.6845444423

In [62]:
def kernel_g(dt, w):
    g = np.multiply(w, np.exp(np.multiply(-w, dt)))
    g[g > 1] = 0
    return g

In [63]:
def kernel_int_g(dt, w):
    G = np.subtract(1, np.exp(np.multiply(-w, dt)))
    G[G < 0] = 0
    return G

In [109]:
def real_err(A, A_m):
    err_1 = np.divide(abs(A - A_m), A)
    err_2 = abs(A - A_m) 
    
    err_1 = np.nan_to_num(err_1)
    err_1 = convert_inf(err_1, np.isinf(err_1))
    
    err = np.multiply(err_1, ((A!=0)*1).astype(float)) + np.multiply(err_2, ((A==0)*1).astype(float))
    err = np.sum(err.flatten())/float(A.size)
    
    return err

In [33]:
def convert_inf(A, inf_A):
    for i in range(0, len(inf_A)):
        for j in range(0, len(inf_A[0])):
            if(inf_A[i][j]):
                A[i][j] = 0
    return A

In [140]:
def optimize_mu(hp_samples, D, w, rho, num_iter_1, num_iter_2, thold, real_A):
#     print (num_iter_1)
#     print (num_iter_2)
    Iteration_Err = np.zeros(((num_iter_2 + 2), (num_iter_1 + 2)))
    A_m = np.random.rand(D,D);
    eigenvalues_m, eigenvectors_m = LA.eig(A_m)
    max_eigenvalue_m = max(np.absolute(eigenvalues_m))
#     print (np.divide(A_m, max_eigenvalue_m))
    A_m = np.dot(0.8, np.divide(A_m, max_eigenvalue_m))
    
    # reshape may required in mu_m
#     print (D)
    mu_m = np.divide(np.random.rand(D, 1), D)
    UL = np.zeros((D, D))
    ZL = np.zeros((D, D))
    US = np.zeros((D, D))
    ZS = np.zeros((D, D))
    
    for i in range(0, num_iter_1 + 1):
        rho = rho * 1.1
        
        for j in range(0, num_iter_2 + 1):
            print ("No. " + str(i + 1) + " outter while iteration | No. " 
                   + str(j + 1) +  " inner while iteration")
            A_m, mu_m, RelErr = update_mu(A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A)
            Iteration_Err[j + 1, i + 1] = RelErr
        
        s, v, d = np.linalg.svd(np.add(A_m, US))
        v = np.subtract(v, thold / rho)
        v[v < 0] = 0
        ZL = s * v * d.T
        UL = UL + np.subtract(A_m, ZL) #changed A_m-ZL to np.subtract
        
        #idk looks good to me lol
        tmp = np.subtract(abs(np.add(A_m, US)), thold / rho) # may have error 
        
        tmp[tmp < 0] = 0
        ZS = (np.multiply(np.sign(np.add(A_m, US)), tmp))
        US = np.add(US, np.subtract(A_m, ZS))
        
    return A_m, mu_m, Iteration_Err

In [171]:
def update_mu (A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A):
    num_samples = len(hp_samples)
    mu_numerator = np.zeros((D, 1))
    
    C = np.zeros((len(A_m), len(A_m[0])))
#     print (rho)
    A_Step = np.add(np.zeros((len(A_m), len(A_m[0]))), 2 * rho)
#     print (A_Step)
    B = np.add(np.add(np.zeros((len(A_m), len(A_m[0]))), np.multiply(rho, np.subtract(UL, ZL))), 
               np.multiply(rho, np.subtract(US, ZS)))
    
    for s in range(0, num_samples):
        cur_hp_samples = hp_samples[s]
        timestamp = [i[0] for i in cur_hp_samples]
        event = [i[1] for i in cur_hp_samples]
        tc = timestamp[len(timestamp) - 1]
        nc = len(event)
        dt = np.subtract(tc, timestamp)
        
        for i in range(0, nc):
            ui = event[i]
            ti = timestamp[i]
            int_g = kernel_int_g(dt, w)
            
            # Todo: modify matrix B (Incomplete)
            # B(ui,:) = B(ui,:) + double(A_m(ui,:)>0).*repmat(int_g(i),[1,D]);
            B[ui,:] = B[ui,:] + np.multiply((A_m[ui,:]>0).astype(float), np.matlib.repmat(int_g[i], 1, D))
            
#             print (B)
            
            pii = []
            pij = []
            ag_arr = []
            
            if (i > 0):
                tj = timestamp[0 : i]
                uj = event[0 : i]
                kn_g = kernel_g(np.subtract(ti, tj), w)
                ag_arr = np.multiply(A_m[uj, ui], kn_g.T)
            
#             print (mu_m[ui] + sum(ag_arr))
            pii = np.divide(mu_m[ui], mu_m[ui] + sum(ag_arr))
            
            if(i > 0):
                pij = np.divide(ag_arr, mu_m[ui] + sum(ag_arr))
                if (len(pij) != 0 and sum(pij) > 0):
                    for j in range(0, len(uj)):
                        uuj = uj[j]
                        C[uuj, ui] = C[uuj, ui] - pij[j]
                        
            mu_numerator[ui] = np.add(mu_numerator[ui], pii)
            
#     print (np.add(np.zeros((D, 1)), tc))
    mu = np.divide(mu_numerator, np.add(np.zeros((D, 1)), tc))
#     print (np.multiply(4, np.multiply(A_Step, C)))
    sqrt_eq = np.sqrt(np.subtract(pow_matrix(B), np.multiply(4 * A_Step, C)))
#     print (np.multiply(2, A_Step))
    A  = np.divide(np.add(-1 * B, sqrt_eq), 2 * A_Step)
#     print (A)
    RelErr = real_err(real_A, A)
    
    print ("non-zero in mu = " + str(np.count_nonzero(mu)))
    print ("non-zero in C = "  + str(np.count_nonzero(C)))
    print ("non-zero in B = "  + str(np.count_nonzero(B)) + ", non-zero in sqrt = " + str(np.count_nonzero(sqrt_eq)))
    print ("real error = " + "{:.4f}".format(RelErr) 
#            + ", correlation = " + "{:.4f}".format()  # (Incomplete)
           + "#non-zero in A = " + str(np.count_nonzero(A)))
            
    A = np.nan_to_num(A)
    A = convert_inf(A, np.isinf(A))
    
    return A, mu, RelErr

In [172]:
def pow_matrix(B_matrix):
    for i in range(0, len(B_matrix)):
        for j in range(0, len(B_matrix[0])):
            B_matrix[i][j] = B_matrix[i][j] ** 2;
    return B_matrix

In [88]:
#start optimization
num_iter_1 = 2;    # Number of Iteration of the First while loop (while k = 1,2 ...)
num_iter_2 = 7;    # Number of Iteration of the Second while loop (while not converge)

rho = 0.09;
thold = 1;         # thershold value

In [173]:
[x,y,Iteration_Err] = optimize_mu(hawkes_process_samples,D,w,rho,num_iter_1,num_iter_2,thold,A);

No. 1 outter while iteration | No. 1 inner while iteration


/Users/queenieli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/queenieli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


non-zero in mu = 997
non-zero in C = 606341
non-zero in B = 997000, non-zero in sqrt = 997000
real error = 1988007.8839#non-zero in A = 997000
No. 1 outter while iteration | No. 2 inner while iteration
non-zero in mu = 997
non-zero in C = 606131
non-zero in B = 1000, non-zero in sqrt = 607075
real error = 143.1167#non-zero in A = 607075
No. 1 outter while iteration | No. 3 inner while iteration


/Users/queenieli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in sqrt


non-zero in mu = 997
non-zero in C = 605662
non-zero in B = 607047, non-zero in sqrt = 607075
real error = nan#non-zero in A = 607075
No. 1 outter while iteration | No. 4 inner while iteration
non-zero in mu = 997
non-zero in C = 606103
non-zero in B = 972, non-zero in sqrt = 607047
real error = 143.0833#non-zero in A = 607047
No. 1 outter while iteration | No. 5 inner while iteration
non-zero in mu = 997
non-zero in C = 605856
non-zero in B = 607047, non-zero in sqrt = 607047
real error = 1247408.5153#non-zero in A = 607047
No. 1 outter while iteration | No. 6 inner while iteration
non-zero in mu = 997
non-zero in C = 606103
non-zero in B = 972, non-zero in sqrt = 607047
real error = 143.0831#non-zero in A = 607047
No. 1 outter while iteration | No. 7 inner while iteration
non-zero in mu = 997
non-zero in C = 605850
non-zero in B = 607047, non-zero in sqrt = 607047
real error = 1247408.5696#non-zero in A = 607047
No. 1 outter while iteration | No. 8 inner while iteration
non-zero in m